# Airfoil Analysis Notebook

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import numpy as np

from ICARUS.core.units import calc_mach, calc_reynolds
from ICARUS.airfoils.airfoil import Airfoil
from ICARUS.core.struct import Struct
from ICARUS.computation.solvers.solver import Solver
from ICARUS.computation.solvers.OpenFoam.files.setup_case import MeshType

# Load DB

In [3]:
from ICARUS.database.db import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)
DB.load_data()

Error loading airfoil images from self.DB2D. Got error: [Errno 2] No such file or directory: 'E:\\Icarus\\Data\\2D\\IMAGES\\images'


In [4]:
print(f"Total number of loaded airfoils {len(list(DB.foils_db.airfoils.keys()))}")
print(
    f"Total number of computed airfoil data {len(list(DB.foils_db._raw_data.keys()))}"
)
print(f"Total number of computed airfoil polars {len(list(DB.foils_db.polars.keys()))}")

Total number of loaded airfoils 4
Total number of computed airfoil data 4
Total number of computed airfoil polars 4


# Reynolds And Mach and AoA

In [5]:
# Assumptions
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 35.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max
print(f"Mach range: {mach_min} - {mach_max}")
print(f"Mach:\n{mach}\n")

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds = np.linspace(reynolds_min, reynolds_max, 20)
# reynolds = [1e5, 10e6]

REYN = reynolds_max
print(f"Reynolds range: {reynolds_min} - {reynolds_max}")
print(f"Re:\n{reynolds}\n")
# ANGLE OF ATTACK SETUP
aoa_max: float = 12
aoa_min: float = -10
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

print(f"Angles of attack:\n{angles}\n")

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

Mach range: 0.029385836027034967 - 0.08815750808110491
Mach:
[0.08815751 0.08162732 0.07509714 0.06856695 0.06203676 0.05550658
 0.04897639 0.04244621 0.03591602 0.02938584]

Reynolds range: 32051.28205128205 - 1121794.8717948718
Re:
[  32051.28205128   89406.20782726  146761.13360324  204116.05937922
  261470.9851552   318825.91093117  376180.83670715  433535.76248313
  490890.68825911  548245.61403509  605600.53981107  662955.46558704
  720310.39136302  777665.317139    835020.24291498  892375.16869096
  949730.09446694 1007085.02024291 1064439.94601889 1121794.87179487]

Angles of attack:
[-10.   -9.5  -9.   -8.5  -8.   -7.5  -7.   -6.5  -6.   -5.5  -5.   -4.5
  -4.   -3.5  -3.   -2.5  -2.   -1.5  -1.   -0.5   0.    0.5   1.    1.5
   2.    2.5   3.    3.5   4.    4.5   5.    5.5   6.    6.5   7.    7.5
   8.    8.5   9.    9.5  10.   10.5  11.   11.5  12. ]



In [6]:
cleaning = False
calcF2W = True
calcOpenFoam = False
calcXFoil = True

# Get Airfoils

In [7]:
all_airfoils = list(DB.foils_db.airfoils.keys())

airfoils_to_compute = [
    airfoil
    for airfoil in all_airfoils
    if (
        # airfoil.upper().startswith("AG")  # or
        # airfoil.upper().startswith('CLARK') or
        # airfoil.upper().startswith('DAE') or
        # airfoil.upper().startswith('E') or
        # airfoil.upper().startswith('H') or
        # airfoil.upper().startswith('M') or
        # airfoil.upper().startswith('N') or
        # airfoil.upper().startswith('O') or
        # airfoil.upper().startswith('W')
    )
]
airfoils_to_compute = ["NACA4415", "NACA4412"]
print(len(airfoils_to_compute))

2


# Xfoil

In [8]:
if calcXFoil:
    from ICARUS.computation.solvers.Xfoil.xfoil import Xfoil

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get airfoil
        airf: Airfoil = DB.get_airfoil(airfoil)
        # airf.plot(scatter=True)
        # airf.repanel(120, distribution="cosine")
        xfoil: Solver = Xfoil()

        # Import Analysis
        analysis: str = xfoil.get_analyses_names()[1]  # Run
        xfoil.select_analysis(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=False)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.mach = MACH
        options.reynolds = reynolds
        options.min_aoa = aoa_min
        options.max_aoa = aoa_max
        options.aoa_step = 0.5

        # Set Solver Options
        solver_parameters.max_iter = 1000
        solver_parameters.Ncrit = 9
        solver_parameters.xtr = (0.1, 0.2)
        solver_parameters.print = False

        xfoil.define_analysis(options, solver_parameters)
        xfoil.print_analysis_options()

        # RUN
        xfoil.execute(parallel=False)
        # Get polar
        polar = DB.foils_db.get_polars(airf.name.upper(), "Xfoil")
        polar.plot()


Running airfoil NACA4415

Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                       | Description                       |
|-----------|-----------------------------|-----------------------------------|
| airfoil   | Complex Datatype (naca4415) | Airfoil Object                    |
| mach      | 0.08815750808110491         | Mach number                       |
| reynolds  | Complex Datatype            | List of Reynold's numbers to run  |
| min_aoa   | -10                         | Minimum angle of attack           |
| max_aoa   | 12                          | Maximum angle of attack           |
| aoa_step  | 0.5                         | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a m

  0%|          | 0/20 [00:00<?, ?it/s]

Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Adding NACA4415 to the database
Analysis Completed

Running airfoil NACA4412

Airfoil Polar Analysis For a multiple Reynolds using aseq
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds using aseq: 

| VarName   | Value                       | Description                       |
|-----------|-----------------------------|-----------------------------------|
| airfoil   | Complex Datatype (naca4412) | Airfoil 

E:\Icarus\ICARUS\airfoils\airfoil_polars.py:429: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend().remove()


  0%|          | 0/20 [00:00<?, ?it/s]

Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Repaneling Airfoil with 100
Adding NACA4412 to the database
Analysis Completed


E:\Icarus\ICARUS\airfoils\airfoil_polars.py:429: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend().remove()


In [9]:
polar = DB.foils_db.get_polars(airfoils_to_compute[1], "Xfoil")
ax = polar.plot()

E:\Icarus\ICARUS\airfoils\airfoil_polars.py:429: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend().remove()


# Foil2Wake

In [10]:
if calcF2W:
    # Import Solver
    from ICARUS.computation.solvers.Foil2Wake.f2w_section import Foil2Wake

    f2w_s: Solver = Foil2Wake()

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # Import Analysis
        analysis: str = f2w_s.get_analyses_names()[1]  # Run
        f2w_s.select_analysis(analysis)

        # Get Options
        options: Struct = f2w_s.get_analysis_options(verbose=False)
        solver_parameters: Struct = f2w_s.get_solver_parameters()

        # Set Options
        options.airfoil = DB.get_airfoil(airfoil)
        options.reynolds = reynolds
        options.mach = MACH
        options.angles = angles

        solver_parameters.f_trip_upper = 0.03
        solver_parameters.f_trip_low = 0.1

        # RUN
        f2w_s.define_analysis(options, solver_parameters)
        f2w_s.print_analysis_options()
        f2w_s.execute(parallel=True)

        # Get Results
        polars = f2w_s.get_results()

TypeError: type 'tqdm' is not subscriptable

In [ ]:
print(airfoils_to_compute[1])
DB.foils_db.polars[airfoils_to_compute[1]]

# Mapflow

In [ ]:
# Not Implemented

# OpenFoam (NOT HIGH MACH)

In [ ]:
if calcOpenFoam:
    airfoils_to_compute: list[str] = [airf.name]  # ["0008"]
    from ICARUS.computation.solvers.OpenFoam.open_foam import OpenFoam

    for airfoil in airfoils_to_compute:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plot()
        open_foam: Solver = OpenFoam()

        # Import Analysis
        analysis: str = open_foam.get_analyses_names()[0]  # Run
        open_foam.select_analysis(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.angles = angles
        options.reynolds = REYN
        options.mach = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type = MeshType.structAirfoilMesher
        solver_parameters.max_iterations = 20
        solver_parameters.silent = False

        # RUN
        open_foam.set_analysis_options(options)
        open_foam.print_analysis_options()
        open_foam.execute()